In [1]:
import sys
import time
import torch
import torch.nn.functional as F
from utils.utils import load_data, mat2tensor, regression_loss
from model.gcn import GCN
from model.gat import GAT
import numpy as np
import dgl

In [43]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

# Load data
features_list, adjM, labels, train_val_test_idx, dl = load_data('iYO844')

fea_list = list()
for features in features_list:
    if type(features) == np.ndarray:
        features = mat2tensor(features).to(device)
    elif type(features) == dict:
        features = {k: mat2tensor(v).to(device) for k, v in features.items()}
    fea_list.append(features)
    
features_list = fea_list

m_dim = features_list[1].shape[1]

# Set train, val, test index
labels = torch.FloatTensor(labels).to(device)
train_idx = train_val_test_idx['train_idx']
train_idx = np.sort(train_idx)
val_idx = train_val_test_idx['val_idx']
val_idx = np.sort(val_idx)
test_idx = train_val_test_idx['test_idx']
test_idx = np.sort(test_idx)

# Build graph
g = dgl.from_scipy(adjM+(adjM.T))
g = dgl.remove_self_loop(g)
g = dgl.add_self_loop(g)
g = g.to(device)

# Set model
num_labels = dl.labels_train['num_labels']

In [3]:
features_list[0]['logits'].shape

torch.Size([384, 1600, 1600, 2])

In [44]:
import torch
import torch.nn as nn
import dgl
from dgl.nn.pytorch import GraphConv
import dgl.function as fn


# (out_channels, kernel_size, stride, padding)
enzyme_conv_archi = [
    # input: 1600x1600x2
    (2, 4, 4, 0),
    # 400x400x2
    (1, 4, 4, 0),
    # 100x100x1
    (1, 4, 4, 0),
    # 25x25x1
]


class Conv2dBlock(nn.Module):
    def __init__(self, in_channels, out_channels, **kwargs):
        super(Conv2dBlock, self).__init__()
        self.conv = nn.Conv2d(in_channels, out_channels, bias=False, **kwargs)
        self.batchnorm = nn.BatchNorm2d(out_channels)
        self.leakyrelu = nn.LeakyReLU(0.1)

    def forward(self, x):
        return self.leakyrelu(self.batchnorm(self.conv(x)))

In [67]:
class GCN(nn.Module):
    def __init__(self,
                 g,
                 m_dim,
                 num_hidden,
                 num_labels,
                 num_layers,
                 activation,
                 dropout):
        super(GCN, self).__init__()
        self.g = g
        
        # Enzyme feature
        ## Conv2d layers for enzyme logits
        self.conv2d_layers = nn.ModuleList()
        
        in_channels = 2
        for x in enzyme_conv_archi:
            self.conv2d_layers.append(Conv2dBlock(in_channels, x[0], kernel_size=x[1], stride=x[2], padding=x[3],))
            in_channels = x[0]
            
        self.conv2d_layers.append(nn.Flatten(start_dim=1))
        self.conv2d_layers.append(nn.Linear(625, 64))
        self.dropout_conv2d = nn.Dropout(dropout)
        
        ## Linear layers for enzyme single
        self.single_linear = nn.Linear(1600, 64)
        
        # fc layers: to make the features of all the nodes become the same dimension  
        in_dims = [257, 257]
        self.fc_list = nn.ModuleList(
            [nn.Linear(in_dim, num_hidden, bias=True) for in_dim in in_dims])
        for fc in self.fc_list:
            nn.init.xavier_normal_(fc.weight, gain=1.414)
        
        # GC layers 
        self.GClayers = nn.ModuleList() 
        ## input layer
        self.GClayers.append(
            GraphConv(num_hidden, num_hidden, activation=activation))

        ## hidden layers
        for i in range(num_layers - 1):
            self.GClayers.append(
                GraphConv(num_hidden, num_hidden, activation=activation))

        ## output layer
        self.GClayers.append(GraphConv(num_hidden, num_labels))
        self.dropout_GC = nn.Dropout(p=dropout)

    def forward(self, features_list):        
        # 1. Enzyme feature
        ## 1.1 logits tensor (input [384, 1600, 1600, 2])]))
        e_feature_logits = features_list[0]['logits'].permute(0, 3, 1, 2)

        for i, layer in enumerate(self.conv2d_layers):
            e_feature_logits = self.dropout_conv2d(e_feature_logits)
            e_feature_logits = layer(e_feature_logits)
        # After: e_feature_logits: [384, 64]
        print(f'e_feature_logits: {e_feature_logits.shape}')
        
        ## 1.2 single representation vector
        e_feature_single = features_list[0]['single']
        e_feature_single = self.single_linear(e_feature_single)
        # After: e_feature_single: [384, 64]
        print(f'e_feature_single: {e_feature_single.shape}')
        
        ## 1.3 concatenate logits and single
        e_feature = torch.cat((e_feature_logits, e_feature_single), 1)
        e_dim = e_feature.shape[1]
        # After: e_feature: [384, 128]
        print(f'e_feature: {e_feature.shape}')
        
        # 2. Pad the features of all the nodes and make them the same dimension
        m_feature = features_list[1]
        
        e_feature = torch.cat((e_feature, torch.zeros((e_feature.shape[0], m_dim))), 1)  # [384, e_dim+129] i.e. [616, 257]
        print(f'e_feature: {e_feature.shape}')
        
        m_feature = torch.cat((torch.zeros((m_feature.shape[0], e_dim)), m_feature), 1)  # [616, 128+m_dim] i.e. [616, 257]
        print(f'm_feature: {m_feature.shape}')
        
        features_list = [e_feature, m_feature]
        
        h = []
        for fc, feature in zip(self.fc_list, features_list):
            h.append(fc(feature))
        h = torch.cat(h, 0) # [1000, hidden_dim]
        
        print(f'h: {h.shape}')
        
        # 3. GC layers
        for i, layer in enumerate(self.GClayers):
            h = self.dropout_GC(h)
            h = layer(self.g, h)
            
        # After: h: [1000, num_labels]

        return h


In [68]:
net = GCN(g, m_dim, 128, num_labels, 3, F.elu, 0.2)

In [69]:
h = net(features_list)
print(h.shape)

e_feature_logits: torch.Size([384, 64])
e_feature_single: torch.Size([384, 64])
e_feature: torch.Size([384, 128])
e_feature: torch.Size([384, 257])
m_feature: torch.Size([616, 257])
h: torch.Size([1000, 128])
torch.Size([1000, 1])


In [18]:
e_s = features_list[0]['single']
print(e_s.shape)

e_s = nn.Linear(1600, 64)(e_s)

print(e_s.shape)


torch.Size([384, 1600])
torch.Size([384, 64])


In [34]:
e_feature = torch.cat((e, e_s), 1)
e_dim = e_feature.shape[1]
print(e_feature.shape)

torch.Size([384, 128])


In [35]:
m_feature = features_list[1]
print(m_feature.shape)

torch.Size([616, 129])


In [36]:
e_feature = torch.cat((e_feature, torch.zeros((e_feature.shape[0], m_dim))), 1)
print(e_feature.shape)

torch.Size([384, 257])


In [37]:
m_feature = torch.cat((torch.zeros((m_feature.shape[0], e_dim)), m_feature), 1)
print(m_feature.shape)

torch.Size([616, 257])


In [38]:
features_list = [e_feature, m_feature]
h = torch.cat(features_list, 0)
h.shape

torch.Size([1000, 257])

In [10]:
e_feature_logits = features_list[0]['logits']
e_feature_logits.shape

torch.Size([384, 1600, 1600, 2])

In [28]:
# net = Conv2dBlock(2, 2, kernel_size=4, stride=4, padding=0,)

In [29]:
e_feature_logits.permute(0, 3, 1, 2).shape

torch.Size([384, 2, 1600, 1600])

In [30]:
e = net(e_feature_logits.permute(0, 3, 1, 2))

In [31]:
e.shape

torch.Size([384, 2, 400, 400])

In [32]:
torch.flatten(e, start_dim=1).shape

torch.Size([384, 320000])

In [2]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

# Load data
features_list, adjM, labels, train_val_test_idx, dl = load_data('iYO844')

In [8]:
fea_list = []

for features in features_list:
    if type(features) == np.ndarray:
        features = mat2tensor(features).to(device)
    elif type(features) == dict:
        features = {k: mat2tensor(v).to(device) for k, v in features.items()}
    fea_list.append(features)
        
fea_list[0]['single']

tensor([[-2.6357, 11.0893,  0.9175,  ...,  0.0000,  0.0000,  0.0000],
        [14.0186, 11.7537, 13.0343,  ...,  0.0000,  0.0000,  0.0000],
        [15.9612, 12.4831,  6.6941,  ...,  0.0000,  0.0000,  0.0000],
        ...,
        [17.3412,  9.8042,  8.6208,  ...,  0.0000,  0.0000,  0.0000],
        [ 2.2316,  7.7314,  8.3776,  ...,  0.0000,  0.0000,  0.0000],
        [13.6403, 13.3657, 12.5303,  ...,  0.0000,  0.0000,  0.0000]])

In [4]:
features_list = [mat2tensor(features).to(device)
                    for features in features_list]

AttributeError: 'dict' object has no attribute 'to'

In [3]:
features_list[0]['single']

array([[-2.6357093, 11.089254 ,  0.9175112, ...,  0.       ,  0.       ,
         0.       ],
       [14.018574 , 11.753677 , 13.034294 , ...,  0.       ,  0.       ,
         0.       ],
       [15.961157 , 12.483061 ,  6.694125 , ...,  0.       ,  0.       ,
         0.       ],
       ...,
       [17.341208 ,  9.80417  ,  8.620768 , ...,  0.       ,  0.       ,
         0.       ],
       [ 2.231642 ,  7.7314253,  8.377593 , ...,  0.       ,  0.       ,
         0.       ],
       [13.640332 , 13.365712 , 12.530339 , ...,  0.       ,  0.       ,
         0.       ]], dtype=float32)

In [2]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

# Load data
features_list, adjM, labels, train_val_test_idx, dl = load_data('iYO844')
features_list = [mat2tensor(features).to(device)
                    for features in features_list]
m_dim = features_list[1].shape[1]

# Set train, val, test index
labels = torch.FloatTensor(labels).to(device)
train_idx = train_val_test_idx['train_idx']
train_idx = np.sort(train_idx)
val_idx = train_val_test_idx['val_idx']
val_idx = np.sort(val_idx)
test_idx = train_val_test_idx['test_idx']
test_idx = np.sort(test_idx)

# Build graph
g = dgl.from_scipy(adjM+(adjM.T))
g = dgl.remove_self_loop(g)
g = dgl.add_self_loop(g)
g = g.to(device)

# Set model
num_labels = dl.labels_train['num_labels']

TypeError: expected np.ndarray (got Series)

In [ ]:
features_list

In [6]:
net = GCN(g, m_dim, 64, num_labels, 2, F.elu, 0.1)
net

GCN(
  (layers): ModuleList(
    (0): GraphConv(in=64, out=64, normalization=both, activation=<function elu at 0x7fa205772940>)
    (1): GraphConv(in=64, out=64, normalization=both, activation=<function elu at 0x7fa205772940>)
    (2): GraphConv(in=64, out=2, normalization=both, activation=None)
  )
  (e_conv): Sequential(
    (0): Conv2dBlock(
      (conv): Conv2d(1, 1, kernel_size=(4, 4), stride=(4, 4), bias=False)
      (batchnorm): BatchNorm2d(1, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (leakyrelu): LeakyReLU(negative_slope=0.1)
    )
    (1): Conv2dBlock(
      (conv): Conv2d(1, 1, kernel_size=(4, 4), stride=(4, 4), bias=False)
      (batchnorm): BatchNorm2d(1, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (leakyrelu): LeakyReLU(negative_slope=0.1)
    )
    (2): Conv2dBlock(
      (conv): Conv2d(1, 1, kernel_size=(4, 4), stride=(4, 4), bias=False)
      (batchnorm): BatchNorm2d(1, eps=1e-05, momentum=0.1, affine=True, track_run

In [7]:
result = net(features_list)

In [8]:
result.shape

torch.Size([1000, 2])

In [9]:
result

tensor([[ 0.5051, -0.6336],
        [ 0.9062, -1.1658],
        [ 0.9253, -1.0670],
        ...,
        [-0.1441, -0.8804],
        [-0.4972,  1.1266],
        [-0.1885, -1.4643]], grad_fn=<AddBackward0>)